# JSON to Vault Converter

In [4]:
import json

json_file = "output.json"

with open(json_file, "r") as file:
    json_f = file.read()
    
# data = json.load(json_f)

In [6]:
data = json.loads(json_f)

In [57]:
import pprint as pp
# pp.pprint(data['sdnList']['sdnEntry'])

entity_data = data['sdnList']['sdnEntry']
print(len(entity_data))
print(type(entity_data))

443
<class 'list'>


In [106]:
test_entity = entity_data[3]
test_entity

{'uid': '9642',
 'firstName': 'Mohammed Jamal',
 'lastName': "NU'MAN ALAEDDIN",
 'sdnType': 'Individual',
 'programList': {'program': 'NS-PLC'},
 'akaList': {'aka': [{'uid': '9156',
    'type': 'a.k.a.',
    'category': 'strong',
    'lastName': 'ALA AL DIN',
    'firstName': 'Mohamad Nouman'},
   {'uid': '9157',
    'type': 'a.k.a.',
    'category': 'strong',
    'lastName': 'AL NATSHEH',
    'firstName': 'Mohammed'}]},
 'addressList': {'address': {'uid': '12129',
   'city': 'Hebron',
   'stateOrProvince': 'West Bank',
   'country': 'Palestinian'}},
 'dateOfBirthList': {'dateOfBirthItem': {'uid': '4702',
   'dateOfBirth': '1959',
   'mainEntry': 'true'}}}

## Individual Extraction

In [312]:

import os

def parse_individual(entity):
    
    """
    Given an XML line of sanction data for an individual,
    extracts useful information and populates a template 
    with YAML formated properties.
    
    Returns X and Y, where X is the file name and Y is the 
    populated template
    """

    ## Combine first and last names to make a file name
    if "firstName" in entity.keys():
        
        if "lastName" in entity.keys():
            file_name = f"{entity["lastName"]}, {entity["firstName"]}"
    
        else:
            file_name = f"{entity["firstName"]}"
    
    
    elif "lastName" in entity.keys():
        file_name = f"{entity["lastName"]}"
        
    else:
        file_name = "wtf mate"
        
    ## Create Program List
    ###### Lets see if this is actually an issue. Will only need to address if there are listing with more than one program.
    # program_list = []
    
    # for key in entity["programList"].keys():
    #     program_list.append(entity["programkey)
    
    program = entity["programList"]["program"]
    
    ## Date of Birth
    if "dateOfBirthList" in entity.keys():
        date_of_birth = entity["dateOfBirthList"]["dateOfBirthItem"]["dateOfBirth"]
    else:
        date_of_birth = ""
    
    ## Place of Birth
    if "placeOfBirthList" in entity.keys():
        place_of_birth = entity["placeOfBirthList"]["placeOfBirthItem"]["placeOfBirth"]
    else:
        place_of_birth = ""
    
    
    indv_note_template = f"""---
Entity Type: Individual
Sanctions Program: [[{program}]]
Place of Birth: {place_of_birth}
Date of Birth: {date_of_birth}
---

"""
    aka_header = """
## AKAs
| Type | Category | Last Name | First Name |
|------|----------|-----------|------------|
"""

    address_header = """
## Addresses
| City | State or Province | Country | 
|------|-------------------|---------|
"""

    if "akaList" in entity.keys():
        
        # Note that the value of akaList may be a list of dictionaries or a single dictionary
        
        if type(entity["akaList"]["aka"]) == list:
            
            for aka in entity["akaList"]["aka"]:
            
                if "firstName" in aka.keys():
                    aka_first_name = aka["firstName"]
                else:
                    aka_first_name = ""
                
                aka_line = ["|", aka["type"] , "|", aka["category"], "|", aka["lastName"], "|", aka_first_name, "|\n"]
                aka_header += " ".join(aka_line)
                
        elif type(entity["akaList"]["aka"]) == dict:
            aka = entity["akaList"]["aka"] 
            
            if "firstName" in aka.keys():
                aka_first_name = aka["firstName"]
            else:
                aka_first_name = ""
                    
            aka_line = ["|", aka["type"], "|", aka["category"], "|", aka["lastName"], "|", aka["firstName"], "|\n"]
            aka_header += " ".join(aka_line)
        
        indv_note_template += aka_header
        indv_note_template += "\n"
            
        
    if "addressList" in entity.keys():
       
       # Note that the value of akaList may be a list of dictionaries or a single dictionary

        if type(entity["addressList"]["address"]) == list:
            

            for address in entity["addressList"]["address"]:
            
            
                if "city" in address.keys():
                    address_city = address["city"]
                else:
                    address_city = ""
                    
                if "stateOrProvince" in address.keys():
                    address_stateprov = address["stateOrProvince"]
                else:
                    address_stateprov = ""
                    
                if "country" in address.keys():
                    address_country = address["country"]
                else:
                    address_country = ""
                
                
                address_line = ["|", address_city, "|", address_stateprov, "|", address_country, "|\n"]
                address_header += " ".join(address_line)
                
                
                
        elif type(entity["addressList"]["address"]) == dict:
            
            
            address = entity["addressList"]["address"] 
            
            
            if "city" in address.keys():
                address_city = address["city"]
            else:
                address_city = ""
                
            if "stateOrProvince" in address.keys():
                address_stateprov = address["stateOrProvince"]
            else:
                address_stateprov = ""
                
            if "country" in address.keys():
                address_country = address["country"]
            else:
                    address_country = ""
                
                    
            address_line = ["|", address_city, "|", address_stateprov, "|", address_country, "|\n"]
            address_header += " ".join(address_line)

    
        indv_note_template += "\n"
        indv_note_template += address_header
        
    return file_name, indv_note_template

## Entity Extraction

In [313]:
def parse_entity(entity):
    
    """
    Given an XML line of sanction data for an individual,
    extracts useful information and populates a template 
    with YAML formated properties.
    
    Returns X and Y, where X is the file name and Y is the 
    populated template
    """
    ## Title and Property information, remarks
    file_name = f"{entity["lastName"]}"
    program = entity["programList"]["program"]
    
    org_note_template = f"""---
Entity Type: Entity
Sanctions Program: {program}
---
"""

    if "remarks" in entity.keys():
        org_note_template += entity["remarks"] 
        org_note_template += "\n"
        
        
    ## Header Templates
    id_header = """
## IDs
| ID Type | ID Number | ID Country |
|---------|-----------|------------|
"""
    
    aka_header = """
## AKAs
| Type | Category | Name      | 
|------|----------|-----------|
"""

    address_header = """
## Addresses
| Line 1 | Line 2 | line 3 | City | Postal Code| Country | 
|--------|--------|--------|------|------------|---------|
"""

    ## Process Headers
    ### ID List Header
    if "idList" in entity.keys():
        
        # Note that the value of akaList may be a list of dictionaries or a single dictionary
        
        idList_val = entity["idList"]["id"]

        if type(idList_val) == list:
            
            for id in idList_val:
            
                if "idType" in id.keys():
                    id_type = id["idType"]
                else:
                    id_type = ""
                
                if "idNumber" in id.keys():
                    id_number = id["idNumber"]
                else:
                    id_number = ""
                    
                if "idCountry" in id.keys():
                    id_country = id["idCountry"]
                else:
                    id_country = ""
                
                
                id_line = ["|", id_type, "|", id_number, "|", id_country, "|\n"]
                id_header += " ".join(id_line)
                
        elif type(idList_val) == dict:
            
            id = idList_val
            
            if "idType" in id.keys():
                id_type = id["idType"]
            else:
                id_type = ""
            
            if "idNumber" in id.keys():
                id_number = id["idNumber"]
            else:
                id_number = ""
                    
            id_line = ["|", id_type, "|", id_number, "|\n"]
            id_header += " ".join(id_line)
        
        org_note_template += id_header
        org_note_template += "\n"
    
    ### AKA List Header
    if "akaList" in entity.keys():
        
        # Note that the value of akaList may be a list of dictionaries or a single dictionary
        
        akaList_val = entity["akaList"]["aka"]
        
        if type(akaList_val) == list:
            
            for aka in akaList_val:
                aka_line = ["|", aka["type"] , "|", aka["category"], "|", aka["lastName"], "|\n"]
                aka_header += " ".join(aka_line)
                
        elif type(akaList_val) == dict:
            
            aka = akaList_val    
            aka_line = ["|", aka["type"], "|", aka["category"], "|", aka["lastName"], "|\n"]
            aka_header += " ".join(aka_line)
        
        org_note_template += aka_header
        org_note_template += "\n"
    
    
    ### Address List Header
    if "addressList" in entity.keys():
        
        # Note that the value of akaList may be a list of dictionaries or a single dictionary
        
        address_val = entity["addressList"]["address"]
        
        if type(address_val) == list:
            
            for address in address_val:
                
                if "address1" in address.keys():
                    address_1 = address["address1"]
                else:
                    address_1 = ""
                    
                if "address2" in address.keys():
                    address_2 = address["address2"]
                else:
                    address_2 = ""
                
                if "address3" in address.keys():
                    address_3 = address["address3"]
                else:
                    address_3 = ""
                
                if "city" in address.keys():
                    address_city = address["city"]
                else:
                    address_city = ""
                    
                if "postalCode" in address.keys():
                    address_postalcode = address["postalCode"]
                else:
                    address_postalcode = ""
                
                if "country" in address.keys():
                    address_country = address["country"]
                else:
                    address_country = ""                    
                
                
                address_line = ["|", address_1, 
                                "|", address_2,
                                "|", address_3,
                                "|", address_city,
                                "|", address_postalcode,
                                "|", address_country, 
                                "|\n"
                                ]
                
                address_header += " ".join(address_line)
                
                
        elif type(address_val) == dict:
            
            address = address_val    
            
            if "address1" in address.keys():
                address_1 = address["address1"]
            else:
                address_1 = ""
                
            if "address2" in address.keys():
                address_2 = address["address2"]
            else:
                address_2 = ""
            
            if "address3" in address.keys():
                address_3 = address["address3"]
            else:
                address_3 = ""
            
            if "city" in address.keys():
                address_city = address["city"]
            else:
                address_city = ""
                
            if "postalCode" in address.keys():
                address_postalcode = address["postalCode"]
            else:
                address_postalcode = ""
            
            if "country" in address.keys():
                address_country = address["country"]
            else:
                address_country = ""                    
            
            
            address_line = ["|", address_1, 
                            "|", address_2,
                            "|", address_3,
                            "|", address_city,
                            "|", address_postalcode,
                            "|", address_country, 
                            "|\n"
                            ]
                
            address_header += " ".join(address_line)
            
            
        org_note_template += address_header
        org_note_template += "\n"
    
    return file_name, org_note_template
    

## Execution

In [314]:
entity_dict_list = []
import os
destination_folder = "Vault"

# if not os.path.exists(destination_folder):
#     os.makedirs(destination_folder)  

def entity_file_conversion(entity, destination_folder):
    
    """
    Given a list of entity data, iterate over to create a file for each entity. 
    First, check if intended destination folder exists and confirms if not. 
    """
    
    # print(entity["sdnType"])        
    
    if entity["sdnType"] == "Individual":
        file_name, file_content = parse_individual(entity)
        


    elif entity["sdnType"] == "Entity":
        file_name, file_content = parse_entity(entity)

    with open(f'{destination_folder}/{file_name}.md', 'w') as f:
        f.write(file_content)    
    

                
        # if entity["sdnType"]

In [315]:
for entity in entity_data:
    entity_file_conversion(entity, destination_folder)

In [293]:
entity_data[3]

{'uid': '9642',
 'firstName': 'Mohammed Jamal',
 'lastName': "NU'MAN ALAEDDIN",
 'sdnType': 'Individual',
 'programList': {'program': 'NS-PLC'},
 'akaList': {'aka': [{'uid': '9156',
    'type': 'a.k.a.',
    'category': 'strong',
    'lastName': 'ALA AL DIN',
    'firstName': 'Mohamad Nouman'},
   {'uid': '9157',
    'type': 'a.k.a.',
    'category': 'strong',
    'lastName': 'AL NATSHEH',
    'firstName': 'Mohammed'}]},
 'addressList': {'address': {'uid': '12129',
   'city': 'Hebron',
   'stateOrProvince': 'West Bank',
   'country': 'Palestinian'}},
 'dateOfBirthList': {'dateOfBirthItem': {'uid': '4702',
   'dateOfBirth': '1959',
   'mainEntry': 'true'}}}